In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [8]:
def PCA_test(matrix, pca):

    reducedMatrixPCA = pca.transform(matrix)

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return reducedDf

In [9]:
def inverse_pca(matrix,pca):
    m = matrix.copy()
    return pca.inverse_transform(m)

In [10]:
# def addLag(dataset, maxlag):
#     dataset_list = [dataset]

#     for l in range(1, maxlag+1):
#         df = dataset.shift(l)
#         df.columns = [c+'_lag_'+str(l) for c in df.columns]
#         dataset_list.append(df)

#     dataset = pd.concat(dataset_list, axis=1).dropna()
#     return dataset

In [11]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [12]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [13]:
def pca_performance(trainmatrix,testmatrix, components):
    rmseList = []
    r2List = []
    for n in components:
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)
        
        network_prediction = inverse_pca(pcaTest,pca)
        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix, network_prediction, multioutput='variance_weighted')
                
        r2List.append(r2Score)
    
    return r2List

In [14]:
def nonlinearperformance(trainmatrix,testmatrix,components, maxlag=12):
    r2List = []
    for n in components:
        print(n)
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)

#         maxlag = 12
        DateColumns = ['Date', 'Hour']
        lagColumns = [c for c in pcaTrain.columns if c not in DateColumns]

        dataset_train = addLag(pcaTrain, maxlag)

        dataset_test = addLag(pcaTest, maxlag)

        X_train = dataset_train.drop(lagColumns , axis = 1)
        X_test = dataset_test.drop(lagColumns , axis = 1)
        y_train = dataset_train[lagColumns]
        y_test = dataset_test[lagColumns]
#         print(X_train.shape)
#         print(X_test.shape)
#         print(y_train.shape)
#         print(y_test.shape)

        rf2 = RandomForestRegressor(random_state = 0, n_estimators=200, 
                                   min_samples_split=10,
                                   min_samples_leaf= 3, 
                                   max_features= 'sqrt',
                                   max_depth= 30, 
                                   bootstrap= True)

        rf2.fit(X_train,y_train)

        pca_prediction = rf2.predict(X_test)

        network_prediction = inverse_pca(pca_prediction,pca)

        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix[maxlag:], network_prediction, \
                           multioutput='variance_weighted')
        
        r2List.append(r2Score)
    return r2List

#### Preparing Data

In [15]:
hub = 'Penn'
tune_hyp_params = False
pca_comps = 24

In [16]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [17]:
# file = '/home/urwa/Documents/Projects/NYU Remote/project/data/JfkVehiceByHour.csv'

In [18]:
data = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [19]:
data = getTimeSeries(data)

In [20]:
data.shape

(8760, 257)

### Train test Split

In [21]:
sep = int(0.75*len(data))
sep

6570

In [22]:
trainData = data[:sep]
testData = data[sep:]

In [23]:
trainData.shape

(6570, 257)

In [24]:
testData.shape

(2190, 257)

In [25]:
trainData.head(2)

DOLocationID     1    2    3    4    5    6    7    8    9    10  ...   254  \
Date       Hour                                                   ...         
2018-01-01 0       0    0    0    4    0    0    1    0    0    0 ...     0   
           1       0    0    0    8    0    0    6    0    0    0 ...     0   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2018-01-01 0       3    1    0    1    0    0    2    7    3  
           1      13    4    3    0    0    1    3    5   13  

[2 rows x 257 columns]

In [26]:
trainmatrix = trainData.values.astype(np.float64)
testmatrix = testData.values.astype(np.float64)

### Best Model Training

### Normalization

In [27]:
scaler, s_train_matrix = standardize(trainmatrix)
s_test_matrix = scaler.transform(testmatrix)

## PCA

In [28]:
pca,pcaTrain = getPCAFeatures(s_train_matrix,n=pca_comps)
pcaTest = PCA_test(s_test_matrix, pca)

In [29]:
pcaTrain.shape, pcaTest.shape

((6570, 24), (2190, 24))

In [30]:
pcaTrain.head(2)

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,1.184801,3.250119,-0.318029,0.355457,0.966918,-0.220305,-1.316832,-1.486838,-1.828185,2.001920,...,-1.628414,-0.953790,-1.365325,-1.801688,1.122124,-0.173366,0.558390,0.018375,-0.663950,1.264672
1,8.997018,3.908343,-1.278405,-1.377469,-4.659586,0.673455,1.274877,0.881627,1.077749,0.061182,...,2.212959,-3.452219,-1.511216,-0.708915,-1.220211,2.059335,-0.300904,1.172455,-1.867683,1.247529


In [31]:
pcaTrain.index = trainData.index
pcaTrain = pcaTrain.reset_index()

pcaTest.index = testData.index
pcaTest = pcaTest.reset_index()

In [32]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [33]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,18/1/1 1:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [34]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [35]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [36]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [37]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [38]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [39]:
extDf = extDf[selected_columns]

In [40]:
print(pcaTrain.shape)
print(pcaTest.shape)
print(extDf.shape)

(6570, 26)
(2190, 26)
(8760, 14)


In [41]:
pcaTrain['Date'] = pd.to_datetime(pcaTrain['Date'])
pcaTest['Date'] = pd.to_datetime(pcaTest['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [42]:
pcaTrain = pd.merge(pcaTrain,extDf, on=['Date', 'Hour'], how='inner')
print(pcaTrain.shape)

pcaTest = pd.merge(pcaTest,extDf, on=['Date', 'Hour'], how='inner')
print(pcaTest.shape)

(6570, 38)
(2190, 38)


### Lag Variables

In [43]:
pcaTrain.columns

Index(['Date', 'Hour', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', 'Dow', 'arrival', 'maxtemp', 'mintemp', 'avgtemp', 'departure',
       'hdd', 'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [44]:
lagColumns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', 'arrival']
# lagColumns = ['1', '2', '3', 'arrival']

DateColumns = ['Date']

targetColumns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24']

In [45]:
maxlag = 12
# DateColumns = ['Date', 'Hour']
# lagColumns = [c for c in pcaTrain.columns if c not in DateColumns]

dataset_train = addLag(pcaTrain, maxlag, lagColumns)

dataset_train.shape

(6558, 338)

In [46]:
dataset_test = addLag(pcaTest, maxlag, lagColumns)
dataset_test.shape

(2178, 338)

### Modelling

In [47]:
X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
y_train = dataset_train[targetColumns]
y_test = dataset_test[targetColumns]

In [48]:
X_train.shape, X_test.shape

((6558, 313), (2178, 313))

In [49]:
y_train.shape, y_test.shape

((6558, 24), (2178, 24))

In [50]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 150, stop = 300, num = 3)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,3]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [150, 225, 300], 'max_features': ['sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [2, 3], 'bootstrap': [True, False]}


In [51]:
if tune_hyp_params:
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, \
                                   cv = 5, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(X_train, y_train)
    print(rf_random.best_params_)

In [52]:
rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                           min_samples_split=3,
                           min_samples_leaf= 2, 
                           max_features= 'sqrt',
                           max_depth= None, 
                           bootstrap= False)

In [53]:
rf2.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=2019, verbose=0, warm_start=False)

In [54]:
rf2.score(X_train,y_train)

0.9334111675189211

In [55]:
rf2.score(X_test,y_test)

0.6193584755026519

### Predict

In [56]:
pca_prediction = rf2.predict(X_test)
pca_prediction.shape

(2178, 24)

In [57]:
network_prediction = inverse_pca(pca_prediction,pca)
network_prediction.shape

(2178, 257)

In [58]:
network_prediction = inverse_standardize(network_prediction, scaler)
network_prediction.shape

(2178, 257)

### Evaluate

In [59]:
get_rmse(testmatrix[maxlag:], network_prediction)

2.253176834625111

In [60]:
r2_score(testmatrix[maxlag:], network_prediction, multioutput='variance_weighted')

0.6877562967359753